In [1]:
"""
Ethereum Price Prediction - Data Loading & Preprocessing (Clean Import Version)
"""

# ============================================================================
# 1. 기본 파이썬 라이브러리
# ============================================================================
import os
import gc
import time
import json
import warnings
from collections import Counter
from datetime import datetime, timedelta

# ============================================================================
# 2. 데이터 처리 / 수학
# ============================================================================
import numpy as np
import pandas as pd
import pandas_ta as ta

from dotenv import load_dotenv

# ============================================================================
# 3. Numba 최적화
# ============================================================================
from numba import jit

# ============================================================================
# 4. 하이퍼파라미터 최적화
# ============================================================================
import optuna

# ============================================================================
# 5. Scikit-learn 관련 - 전처리, 모델, 평가
# ============================================================================
# 전처리
from sklearn.preprocessing import RobustScaler, StandardScaler

# 특징 선택
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)

# 모델 선택용 툴
from sklearn.model_selection import GridSearchCV

# 지도학습 모델 - 기본 모델
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# 앙상블 모델
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier,
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error
)

# ============================================================================
# 6. 부스팅 라이브러리 (LightGBM / XGBoost / CatBoost)
# ============================================================================
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

# ============================================================================
# 7. TensorFlow / Keras 딥러닝
# ============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Flatten, Activation,
    LSTM, GRU, SimpleRNN, Bidirectional,
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    BatchNormalization, LayerNormalization,
    Attention, MultiHeadAttention,
    Concatenate, Add, Multiply, Lambda,
    Reshape, Permute, RepeatVector, TimeDistributed
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================================================
# 8. PyTorch (선택적)
# ============================================================================
try:
    import torch
    import torch.nn as nn
except ImportError:
    pass

# ============================================================================
# 9. 시계열 분석 도구 (Statsmodels)
# ============================================================================
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# ============================================================================
# 1. 라이브러리 임포트 및 환경 설정
# ============================================================================
# ============================================================================
# 1. 라이브러리 임포트 및 환경 설정
# ============================================================================
import os
import gc
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pandas_ta as ta

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
warnings.filterwarnings('ignore')

# ============================================================================
# 2. 설정 (Configuration)
# ============================================================================
class Config:
    # 경로 설정 (현재 경로 확인 필수)
    DATA_DIR = './macro_data/macro_data' 
    
    # 날짜 설정
    TRAIN_START_DATE = pd.to_datetime('2020-01-01')
    LOOKBACK_DAYS = 200
    
    # 데이터 소스 정의
    DATA_SOURCES = [
        # [CHECK] 파일이 실제로 존재하는지 확인해주세요.
        {'file': 'binance_daily_crypto_data.csv', 'prefix': None}, # 김프용 바이낸스 데이터
        {'file': 'fred_macro_daily.csv',  'prefix': None},          # FRED 매크로 데이터
        
        {'file': 'macro_crypto_data.csv', 'prefix': None},        
        {'file': 'SP500.csv',             'prefix': 'sp500'},
        {'file': 'VIX.csv',               'prefix': 'vix'},
        {'file': 'GOLD.csv',              'prefix': 'gold'},
        {'file': 'DXY.csv',               'prefix': 'dxy'},
        {'file': 'fear_greed.csv',        'prefix': 'fg'},
        {'file': 'eth_funding_rate.csv',  'prefix': 'funding'},
        {'file': 'usdt_eth_mcap.csv',     'prefix': 'usdt'},
        {'file': 'aave_eth_tvl.csv',      'prefix': 'aave'},
        {'file': 'lido_eth_tvl.csv',      'prefix': 'lido'},
        {'file': 'makerdao_eth_tvl.csv',  'prefix': 'makerdao'},
        {'file': 'uniswap_eth_tvl.csv',   'prefix': 'uniswap'},
        {'file': 'curve-dex_eth_tvl.csv', 'prefix': 'curve'},
        {'file': 'eth_chain_tvl.csv',     'prefix': 'chain'},
        {'file': 'layer2_tvl.csv',        'prefix': 'l2'},
    ]

# ============================================================================
# 3. 유틸리티 함수 (Utils)
# ============================================================================
def standardize_date_column(df, filename):
    """날짜 컬럼 표준화: 이름 통일, 포맷 변환, Timezone 제거"""
    # 날짜 컬럼 후보 찾기 (date, timestamp 대소문자 구분 없이)
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'timestamp' in col.lower()]
    
    if not date_cols:
        print(f"  -> [Warning] {filename}: 날짜 컬럼을 찾을 수 없습니다. (Columns: {df.columns.tolist()})")
        return df
    
    date_col = date_cols[0]
    if date_col != 'date':
        df.rename(columns={date_col: 'date'}, inplace=True)
    
    # 날짜 변환
    if filename == 'eth_onchain.csv':
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)
    
    # 결측치 및 Timezone 처리
    df = df.dropna(subset=['date'])
    df['date'] = df['date'].dt.normalize() # 시간 제거 (00:00:00)
    
    if pd.api.types.is_datetime64tz_dtype(df['date']):
        df['date'] = df['date'].dt.tz_convert(None)
    else:
        df['date'] = df['date'].dt.tz_localize(None)
        
    return df

def load_csv(directory, filename):
    """CSV 파일 로드 및 전처리"""
    filepath = os.path.join(directory, filename)
    
    if not os.path.exists(filepath):
        print(f"[Skip] {filename} 파일을 찾을 수 없습니다. (경로: {filepath})")
        return pd.DataFrame()
        
    try:
        df = pd.read_csv(filepath)
        if df.empty:
            print(f"  -> [Warning] {filename} 파일은 비어있습니다.")
            return df
            
        df = standardize_date_column(df, filename)
        return df
        
    except Exception as e:
        print(f"  -> [Error] {filename} 로드 중 오류 발생: {e}")
        return pd.DataFrame()

def add_prefix(df, prefix):
    """컬럼명 접두사 추가"""
    if df.empty or prefix is None:
        return df
    df.columns = [f"{prefix}_{col}" if col != 'date' else col for col in df.columns]
    return df

def smart_fill_missing(df):
    """결측치 처리: 0 -> ffill -> 0"""
    processed_df = df.copy()
    total_cols = len(processed_df.columns)
    
    for i, col in enumerate(processed_df.columns):
        if col == 'date' or processed_df[col].isnull().sum() == 0:
            continue
            
        first_valid_idx = processed_df[col].first_valid_index()
        
        if first_valid_idx is None:
            # 데이터가 아예 없는 컬럼
            processed_df[col] = processed_df[col].fillna(0)
            continue
            
        first_date = processed_df.loc[first_valid_idx, 'date']
        
        # 마스크 생성
        before_mask = processed_df['date'] < first_date
        after_mask = processed_df['date'] >= first_date
        
        # 1. 데이터 시작 전: 0 채움
        processed_df.loc[before_mask, col] = processed_df.loc[before_mask, col].fillna(0)
        # 2. 데이터 시작 후: Forward Fill
        processed_df.loc[after_mask, col] = processed_df.loc[after_mask, col].fillna(method='ffill')
        # 3. 그래도 남은 것(끝부분 등): 0 채움
        processed_df.loc[after_mask, col] = processed_df.loc[after_mask, col].fillna(0)
        
    return processed_df

def print_feature_summary(df):
    """데이터프레임 요약 출력"""
    print(f"\nFeature groups Summary:")
    groups = {
        "Crypto Prices": ['BTC_', 'ETH_', 'BNB_', 'XRP_', 'SOL_', 'ADA_', 'AVAX_', 'DOT_'],
        "Binance (KimP)": ['_binance_'],
        "On-chain": ['eth_'],
        "DeFi TVL": ['aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_'],
        "Layer 2": ['l2_'],
        "Sentiment": ['sentiment', 'news', 'bull_bear'],
        "Macro": ['sp500_', 'vix_', 'gold_', 'dxy_', 'fred_'], 
        "Fear & Greed": ['fg_'],
        "Funding Rate": ['funding_'],
        "Stablecoin": ['usdt_']
    }
    
    total_features = 0
    for name, keywords in groups.items():
        count = len([c for c in df.columns if any(k in c for k in keywords)])
        print(f"  - {name.ljust(15)}: {count} features")
        total_features += count
        
    print(f"  - {'Other'.ljust(15)}: {len(df.columns) - 1 - total_features} features")

# ============================================================================
# 4. 메인 실행 로직
# ============================================================================
print("="*80)
print("DATA PIPELINE START")
print("="*80)

# 1. 데이터 로드
loaded_dfs = []
valid_dates = []

print(f"Loading data from: {Config.DATA_DIR}")

for source in Config.DATA_SOURCES:
    print(f"Processing {source['file']}...", end=" ")
    
    df = load_csv(Config.DATA_DIR, source['file'])
    
    if not df.empty:
        if 'date' not in df.columns:
            print(f"-> [Skip] 'date' column missing.")
            continue
            
        df = add_prefix(df, source['prefix'])
        loaded_dfs.append(df)
        
        max_date = df['date'].max()
        valid_dates.append(max_date)
        print(f"-> OK ({len(df)} rows, ~{max_date.date()})")
    else:
        print("-> Failed or Empty")

print(f"\nSuccessfully loaded {len(loaded_dfs)} files.")

end_date = min(valid_dates) if valid_dates else pd.Timestamp.today()
start_date = Config.TRAIN_START_DATE - timedelta(days=Config.LOOKBACK_DAYS)

print(f"\nTarget Date Range: {start_date.date()} ~ {end_date.date()}")

date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_merged = pd.DataFrame(date_range, columns=['date'])

print("Merging dataframes...")
for df in loaded_dfs:
    df_merged = pd.merge(df_merged, df, on='date', how='left')

print(f"Merged Shape: {df_merged.shape}")
print(f"Missing before fill: {df_merged.isnull().sum().sum():,}")

# 3. 결측치 처리
print("\nRunning Smart Missing Value Imputation...")
df_merged = smart_fill_missing(df_merged)

# 4. 불필요한 컬럼 제거 (Lookback 기간에 데이터가 하나도 없는 컬럼)
lookback_mask = df_merged['date'] < Config.TRAIN_START_DATE
lookback_df = df_merged[lookback_mask]

cols_to_drop = [
    col for col in lookback_df.columns 
    if lookback_df[col].isnull().all() and col != 'date'
]

if cols_to_drop:
    print(f"Dropping {len(cols_to_drop)} fully missing columns in lookback period.")
    df_merged = df_merged.drop(columns=cols_to_drop)

# 5. 최종 결과 확인
df_final = df_merged

print("="*80)
print("PIPELINE COMPLETE")
print("="*80)
print(f"Final Shape: {df_final.shape}")
print(f"Date Period: {df_final['date'].min().date()} ~ {df_final['date'].max().date()}")
print(f"Total Missing: {df_final.isnull().sum().sum()}")

print_feature_summary(df_final)




2025-12-03 00:04:29.036308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-03 00:04:29.036352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-03 00:04:29.037702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 00:04:29.044725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 00:04:29.843363: W tensorflow/comp

DATA PIPELINE START
Loading data from: ./macro_data/macro_data
Processing binance_daily_crypto_data.csv... -> OK (3030 rows, ~2025-12-02)
Processing fred_macro_daily.csv... -> OK (3254 rows, ~2025-12-02)
Processing macro_crypto_data.csv... -> OK (2991 rows, ~2025-12-02)
Processing SP500.csv... -> OK (2241 rows, ~2025-12-01)
Processing VIX.csv... -> OK (2242 rows, ~2025-12-02)
Processing GOLD.csv... -> OK (2239 rows, ~2025-12-02)
Processing DXY.csv... -> OK (2244 rows, ~2025-12-02)
Processing fear_greed.csv... -> OK (2858 rows, ~2025-12-02)
Processing eth_funding_rate.csv... -> OK (2198 rows, ~2025-12-02)
Processing usdt_eth_mcap.csv... -> OK (2926 rows, ~2025-12-02)
Processing aave_eth_tvl.csv... -> OK (2024 rows, ~2025-12-02)
Processing lido_eth_tvl.csv... -> OK (1810 rows, ~2025-12-02)
Processing makerdao_eth_tvl.csv... -> OK (2526 rows, ~2025-12-02)
Processing uniswap_eth_tvl.csv... -> OK (2588 rows, ~2025-12-02)
Processing curve-dex_eth_tvl.csv... -> OK (2118 rows, ~2025-12-02)
Pro

In [2]:
df_merged=df_final.copy()
df_merged.head(10)


date  BTC_binance_Open  BTC_binance_High  BTC_binance_Low  \
0 2019-06-15           8650.88           8864.99          8567.63   
1 2019-06-16           8810.77           9333.00          8760.00   
2 2019-06-17           8953.00           9444.00          8950.00   
3 2019-06-18           9312.13           9336.36          8950.00   
4 2019-06-19           9081.97           9304.00          8960.00   
5 2019-06-20           9253.76           9590.00          9175.20   
6 2019-06-21           9518.06          10174.99          9518.06   
7 2019-06-22          10159.86          11160.00          9921.72   
8 2019-06-23          10729.00          11392.64         10555.18   
9 2019-06-24          10905.95          11143.63         10620.80   

   BTC_binance_Close  BTC_binance_Volume  ETH_binance_Open  ETH_binance_High  \
0            8808.70        31791.636039            262.48            272.27   
1            8953.33        63289.251219            267.78            277.06   
2            9313.96        47895.485374            267.73            276.00   
3            9081.55        51554.569401            273.14            273.58   
4            9255.49        32147.706495            264.68            270.00   
5            9517.12        34556.053982            268.16            273.63   
6           10159.86        50484.415892            271.27            295.94   
7           10729.50       104169.447976            294.25            314.39   
8           10906.07        57445.323945            310.70            321.70   
9           11056.59        43101.045285            308.42            314.41   

   ETH_binance_Low  ETH_binance_Close  ...  aave_aave_eth_tvl  \
0           260.61             267.82  ...                0.0   
1           263.31             267.73  ...                0.0   
2           266.81             273.16  ...                0.0   
3           261.45             264.66  ...                0.0   
4           263.59             268.14  ...                0.0   
5           265.71             271.22  ...                0.0   
6           271.22             294.35  ...                0.0   
7           289.69             310.79  ...                0.0   
8           305.58             308.52  ...                0.0   
9           298.00             311.64  ...                0.0   

   lido_lido_eth_tvl  makerdao_makerdao_eth_tvl  uniswap_uniswap_eth_tvl  \
0                0.0                  419465215                 18302293   
1                0.0                  429759407                 18309591   
2                0.0                  443207073                 17769474   
3                0.0                  445751775                 18014411   
4                0.0                  438027353                 17393056   
5                0.0                  436013887                 17717958   
6                0.0                  436952496                 17850244   
7                0.0                  470581244                 18959534   
8                0.0                  497501560                 19523606   
9                0.0                  495655481                 19194229   

   curve_curve-dex_eth_tvl  chain_eth_chain_tvl  l2_arbitrum_tvl  \
0                      0.0            484243010              0.0   
1                      0.0            495664501              0.0   
2                      0.0            509130301              0.0   
3                      0.0            513188682              0.0   
4                      0.0            504467906              0.0   
5                      0.0            504383890              0.0   
6                      0.0            505731952              0.0   
7                      0.0            543938953              0.0   
8                      0.0            573222685              0.0   
9                      0.0            571499683              0.0   

   l2_optimism_tvl  l2_base_tvl  l2_zksync era_tvl  
0           

In [3]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from numba import jit
import os
from datetime import datetime

# ============================================================================
# 1. 기본 유틸리티
# ============================================================================
def add_indicator_to_df(df_ta, indicator):
    """pandas_ta 지표 결과를 DataFrame에 안전하게 추가"""
    if indicator is None:
        return

    if isinstance(indicator, pd.DataFrame) and not indicator.empty:
        for col in indicator.columns:
            df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series) and not indicator.empty:
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    """지표 생성 시 오류 방지를 위한 래퍼 함수"""
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except Exception as e:
        # func_name = func.__name__ if hasattr(func, '__name__') else str(func)
        # print(f"    ⚠ {func_name} 생성 실패: {str(e)[:50]}")
        return False

# ============================================================================
# 2. 핵심 기술적 지표 계산 (Garman-Klass 추가됨)
# ============================================================================
def calculate_technical_indicators(df):
    # 날짜 정렬 및 인덱스 초기화 (필수)
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    # 기본 데이터 정의 (없으면 에러 방지)
    if 'ETH_Close' not in df.columns:
        print("Error: 'ETH_Close' column is missing.")
        return df
        
    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    open_ = df.get('ETH_Open', close)
    volume = df.get('ETH_Volume', pd.Series(1, index=df.index)) # 거래량 없으면 1로 채움

    try:
        # [NEW] Garman-Klass Volatility (고급 변동성 지표)
        # 양봉/음봉 뿐만 아니라 장중 변동폭(High-Low)까지 고려
        log_hl = (np.log(high / low)) ** 2
        log_co = (np.log(close / open_)) ** 2
        df_ta['GARMAN_KLASS_VOL'] = 0.5 * log_hl - (2 * np.log(2) - 1) * log_co

        # ===== MOMENTUM =====
        df_ta['RSI_14'] = ta.rsi(close, length=14)
        safe_add(df_ta, ta.macd, close, fast=12, slow=26, signal=9)
        safe_add(df_ta, ta.stoch, high, low, close, k=14, d=3)
        df_ta['WILLR_14'] = ta.willr(high, low, close, length=14)
        df_ta['ROC_10'] = ta.roc(close, length=10)
        df_ta['CCI_14'] = ta.cci(high, low, close, length=14)
        safe_add(df_ta, ta.tsi, close, fast=13, slow=25, signal=13)

        # ===== OVERLAP =====
        # SMA/EMA는 필수
        df_ta['SMA_20'] = ta.sma(close, length=20)
        df_ta['SMA_50'] = ta.sma(close, length=50)
        df_ta['EMA_12'] = ta.ema(close, length=12)
        df_ta['EMA_26'] = ta.ema(close, length=26)
        
        # VWMA (거래량 가중)
        df_ta['VWMA_20'] = ta.vwma(close, volume, length=20)

        # ===== VOLATILITY =====
        safe_add(df_ta, ta.bbands, close, length=20, std=2)
        
        # [중요] ATR은 타겟 생성에 필수이므로 safe_add 대신 직접 할당 시도
        try:
            atr_val = ta.atr(high, low, close, length=14)
            if atr_val is not None:
                 df_ta['ATR_14'] = atr_val
            else:
                 # 실패시 대안: 고가-저가 평균
                 df_ta['ATR_14'] = (high - low).rolling(14).mean()
        except:
            df_ta['ATR_14'] = (high - low).rolling(14).mean()

        safe_add(df_ta, ta.kc, high, low, close, length=20)

        # ===== VOLUME =====
        df_ta['OBV'] = ta.obv(close, volume)
        df_ta['MFI_14'] = ta.mfi(high, low, close, volume, length=14)
        df_ta['CMF_20'] = ta.cmf(high, low, close, volume, length=20)
        safe_add(df_ta, ta.eom, high, low, close, volume, length=14)

        # ===== TREND =====
        safe_add(df_ta, ta.adx, high, low, close, length=14)
        safe_add(df_ta, ta.vortex, high, low, close, length=14)

        # ===== 파생 지표 (Custom) =====
        # 1. 이격도 (Disparity)
        df_ta['DISPARITY_SMA_20'] = close / (df_ta['SMA_20'] + 1e-9) - 1
        
        # 2. 골든크로스/데드크로스 시그널
        df_ta['CROSS_SMA_20_50'] = np.where(df_ta['SMA_20'] > df_ta['SMA_50'], 1, -1)
        
        # 3. 볼린저 밴드 폭 (Squeeze 포착용)
        if 'BBU_20_2.0' in df_ta.columns and 'BBL_20_2.0' in df_ta.columns:
             df_ta['BB_WIDTH'] = (df_ta['BBU_20_2.0'] - df_ta['BBL_20_2.0']) / (df_ta['BBM_20_2.0'] + 1e-9)
             df_ta['BB_PCT_B'] = (close - df_ta['BBL_20_2.0']) / (df_ta['BBU_20_2.0'] - df_ta['BBL_20_2.0'] + 1e-9)

        # 4. RSI 과매수/과매도 상태 (One-hot)
        if 'RSI_14' in df_ta.columns:
            df_ta['RSI_OVER_70'] = (df_ta['RSI_14'] > 70).astype(int)
            df_ta['RSI_UNDER_30'] = (df_ta['RSI_14'] < 30).astype(int)

    except Exception as e:
        print(f"\n❌ Technical Indicators Error: {e}")

    return df_ta

# ============================================================================
# 3. 크로스 에셋 & 상관관계 분석
# ============================================================================
def add_enhanced_cross_crypto_features(df):
    df_enhanced = df.copy()
    
    # 기본 수익률 계산 (없으면 계산)
    if 'eth_return' not in df_enhanced.columns:
        df_enhanced['eth_return'] = df['ETH_Close'].pct_change()
        
    # BTC 데이터가 있을 경우에만 실행
    if 'BTC_Close' in df.columns:
        df_enhanced['btc_return'] = df['BTC_Close'].pct_change()
        
        # 1. 상관관계 (Rolling Correlation) - 시장 동조화 수준 파악
        for window in [14, 30, 60]:
            df_enhanced[f'corr_eth_btc_{window}d'] = (
                df_enhanced['eth_return'].rolling(window).corr(df_enhanced['btc_return'])
            ).fillna(0)
            
        # 2. 상대 강도 (Relative Strength) - 누가 더 센가?
        # 양수면 ETH가 더 강함, 음수면 BTC가 더 강함
        df_enhanced['rel_strength_eth_btc'] = df_enhanced['eth_return'] - df_enhanced['btc_return']
        
        # 3. 베타 (Beta) - BTC 대비 ETH의 민감도
        cov = df_enhanced['eth_return'].rolling(30).cov(df_enhanced['btc_return'])
        var = df_enhanced['btc_return'].rolling(30).var()
        df_enhanced['beta_eth_btc_30d'] = (cov / (var + 1e-9)).fillna(1) # 기본값 1

    return df_enhanced

# ============================================================================
# 4. 시차(Lag) 및 변동성 피처 통합 (중복 제거됨)
# ============================================================================
def add_lag_and_volatility_features(df):
    df_proc = df.copy()
    
    # 1. 가격 Lag (핵심 가격 정보만)
    for lag in [1, 2, 3, 7]:
        df_proc[f'eth_return_lag{lag}'] = df_proc['eth_return'].shift(lag)
        
    # 2. 로그 수익률 (정규 분포에 더 가까움)
    df_proc['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    
    # 3. 변동성 (Volatility)
    # 단기(7일) vs 장기(30일) 변동성 비율 -> 추세 전환 시그널
    vol_7 = df_proc['eth_log_return'].rolling(7).std()
    vol_30 = df_proc['eth_log_return'].rolling(30).std()
    df_proc['vol_ratio_7_30'] = vol_7 / (vol_30 + 1e-9)
    
    # 4. 거래량 변화율
    if 'ETH_Volume' in df.columns:
        df_proc['volume_change'] = df['ETH_Volume'].pct_change()
        df_proc['volume_ma_ratio'] = df['ETH_Volume'] / (df['ETH_Volume'].rolling(20).mean() + 1e-9)

    return df_proc

# ============================================================================
# 5. 비정상성 데이터 전처리 (Non-Stationary Handling)
# ============================================================================
def preprocess_non_stationary_features(df):
    """
    가격, TVL 등 절대값으로 커지는 데이터를 비율/변화율로 변환
    """
    df_proc = df.copy()
    
    # 변환할 컬럼들의 접두사 (Prefix)
    prefixes = ['eth_', 'btc_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 
                'chain_', 'l2_', 'sp500_', 'gold_', 'dxy_', 'vix_', 'usdt_']
    
    # 제외할 컬럼 (이미 비율이거나 정규화된 것들)
    exclude_keywords = ['_return', '_pct', '_ratio', '_corr', '_beta', 'rsi_', 'cci_', 'adx_']
    
    cols_to_drop = []
    
    for col in df_proc.columns:
        # 1. 지정된 접두사로 시작하고
        if any(col.lower().startswith(p) for p in prefixes):
            # 2. 이미 변환된 컬럼이 아니라면
            if not any(k in col.lower() for k in exclude_keywords):
                
                # 변화율 (Returns) 생성
                df_proc[f'{col}_pct_change'] = df_proc[col].pct_change().fillna(0)
                
                # 이동평균 괴리율 (Trend Deviation) 생성
                ma_30 = df_proc[col].rolling(30).mean()
                df_proc[f'{col}_trend_ratio'] = (df_proc[col] / (ma_30 + 1e-9) - 1).fillna(0)
                
                if col not in ['ETH_Close', 'date']: # Close와 Date는 살려둠
                     cols_to_drop.append(col)

    df_proc.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    
    # 무한대/NaN 처리
    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df_proc

# ============================================================================
# 6. 타겟 생성 (Triple Barrier Method) - Numba 가속
# ============================================================================
@jit(nopython=True)
def compute_triple_barrier_targets(close, high, low, atr, lookahead, tp_mult, sl_mult):
    n = len(close)
    targets = np.zeros(n, dtype=np.int32) # 0: Hold/Neutral, 1: Buy/Profit
    
    for i in range(n - lookahead):
        entry_price = close[i]
        volatility = atr[i]
        
        if volatility <= 0: continue # ATR 오류 방지
            
        # TP(익절) / SL(손절) 라인 설정
        tp_price = entry_price + (volatility * tp_mult)
        sl_price = entry_price - (volatility * sl_mult)
        
        label = 0 # 기본: 아무것도 안 찍힘 (횡보)
        
        # 미래 N개 캔들 확인
        for j in range(1, lookahead + 1):
            future_high = high[i + j]
            future_low = low[i + j]
            
            if future_low <= sl_price:
                label = 0 # Stop Loss Hit
                break
            elif future_high >= tp_price:
                label = 1 # Take Profit Hit
                break
                
        targets[i] = label
        
    return targets

def create_targets(df, lookahead=5, tp_mult=2.0, sl_mult=1.0):
    """
    머신러닝 학습용 타겟 생성
    """
    df_target = df.copy()
    
    # 필수 데이터 확인
    required_cols = ['ETH_Close', 'ETH_High', 'ETH_Low', 'ATR_14']
    for col in required_cols:
        if col not in df_target.columns:
            print(f"Error: '{col}' missing for target creation.")
            return df_target

    # 데이터 준비 (Numpy array 변환)
    close = df_target['ETH_Close'].values
    high = df_target['ETH_High'].values
    low = df_target['ETH_Low'].values
    atr = df_target['ATR_14'].fillna(method='ffill').values # 결측치 있으면 안됨
    
    # 1. Triple Barrier Target (분류 문제용)
    targets = compute_triple_barrier_targets(
        close, high, low, atr, lookahead, tp_mult, sl_mult
    )
    df_target['target_triple_barrier'] = targets
    
    df_target['target_next_return'] = np.log(df_target['ETH_Close'].shift(-1) / df_target['ETH_Close'])
    
    # 3. Next Close (참고용)
    df_target['target_next_close'] = df_target['ETH_Close'].shift(-1)
    
    return df_target

# ============================================================================
# 7. 결측치 최종 처리 (Paper Based)
# ============================================================================
def handle_missing_values_final(df):
    df_clean = df.copy()
    
    # 1. 무한대(inf) 제거
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    
    # 2. Forward Fill (과거 데이터로 채움 - 시계열에 적합)
    df_clean = df_clean.fillna(method='ffill')
    
    # 3. 그래도 남은 앞부분 결측치(데이터 시작 전)는 0으로 채움
    df_clean = df_clean.fillna(0)
    
    return df_clean

# ============================================================================
# [NEW] 거시경제(Macro) 데이터 가공
# ============================================================================
def process_macro_features(df):
    """
    CPI, 금리, 환율 등 매크로 데이터를 변화율 및 이격도로 변환
    """
    df_macro = df.copy()
    
    # 1. FRED CPI (fred_CPI) - 전년 대비 변화율 (YoY)
    if 'fred_CPI' in df_macro.columns:
        # CPI는 지수이므로 변화율(인플레이션율)이 중요
        df_macro['macro_cpi_yoy'] = df_macro['fred_CPI'].pct_change(365) # 1년 전 대비
        df_macro['macro_cpi_mom'] = df_macro['fred_CPI'].pct_change(30)  # 1달 전 대비
        
    # 2. 금리 (fred_EFFR, fred_DGS10) - 절대 레벨 및 장단기 금리차
    if 'fred_DGS10' in df_macro.columns and 'fred_EFFR' in df_macro.columns:
        df_macro['macro_yield_curve'] = df_macro['fred_DGS10'] - df_macro['fred_EFFR'] # 장단기 금리차 (경기침체 신호)
        
    # 3. M2 통화량 (fred_M2SL) - 유동성 증감
    if 'fred_M2SL' in df_macro.columns:
        df_macro['macro_m2_growth'] = df_macro['fred_M2SL'].pct_change(30) # 월간 유동성 변화
        
    # 4. 환율 (fred_DEXKOUS) - 변동성
    if 'fred_DEXKOUS' in df_macro.columns:
        df_macro['macro_fx_volatility'] = df_macro['fred_DEXKOUS'].pct_change().rolling(30).std()

    return df_macro

# ============================================================================
# [NEW] 온체인(On-chain) 데이터 가공
# ============================================================================
def process_onchain_features(df):
    """
    TVL, 거래량, 활성 주소 등 온체인 데이터를 시그널로 변환
    """
    df_chain = df.copy()
    
    # 온체인 데이터 컬럼들 (접두어로 식별)
    # eth_ (온체인), aave_, lido_, makerdao_ (DeFi TVL), usdt_ (스테이블코인)
    onchain_prefixes = ['eth_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_', 'l2_', 'usdt_']
    
    for col in df_chain.columns:
        # 온체인 데이터이면서, 가격 데이터가 아닌 경우
        if any(col.startswith(p) for p in onchain_prefixes) and 'Close' not in col and 'price' not in col:
            
            # 1. 단기 급등락 시그널 (Z-score)
            # 평소보다 TVL이나 활성 주소가 급격히 늘어나면 호재/악재
            mean_30 = df_chain[col].rolling(30).mean()
            std_30 = df_chain[col].rolling(30).std()
            df_chain[f'{col}_zscore'] = (df_chain[col] - mean_30) / (std_30 + 1e-9)
            
            # 2. 추세 (7일 이동평균선 대비 현재 위치)
            df_chain[f'{col}_trend'] = df_chain[col] / (df_chain[col].rolling(7).mean() + 1e-9) - 1

    # [Specific] 스테이블코인(USDT) 시가총액 증가율 = 잠재 매수 대기 자금
    if 'usdt_Marketcap' in df_chain.columns: # 컬럼명 확인 필요
        df_chain['onchain_stable_inflow'] = df_chain['usdt_Marketcap'].pct_change(7)
        
    return df_chain

# ============================================================================
# [NEW] 김치 프리미엄(Kim-P) 심화 가공
# ============================================================================
def process_kimp_features(df):
    """
    이미 생성된 김프(implied_fx, premium_pct) 데이터를 기반으로
    매매 시그널(역추세/추세) 생성
    """
    df_kimp = df.copy()
    
    # 앞 단계에서 'BTC_premium_pct' 등이 생성되었다고 가정
    kimp_cols = [c for c in df_kimp.columns if 'premium_pct' in c]
    
    for col in kimp_cols:
        # 1. 김프 역추세 전략 (김프가 너무 높으면 매도 신호)
        # 역사적(90일) 고점 구간인지?
        rolling_max = df_kimp[col].rolling(90).max()
        df_kimp[f'{col}_is_high'] = (df_kimp[col] >= rolling_max * 0.95).astype(int)
        
        # 2. 김프 급락 (해외 거래소 폭등 or 국내 패닉셀)
        df_kimp[f'{col}_change'] = df_kimp[col].diff()
        
    return df_kimp

def preprocess_non_stationary_features(df):
    """
    가격, TVL 등 절대값(Non-Stationary) 데이터를 비율/변화율(Stationary)로 변환하고,
    원본 절대값 컬럼은 학습 데이터에서 과감히 삭제합니다.
    
    [NEW] 바이낸스(_binance_) 가격 데이터도 삭제 대상에 포함
    """
    df_proc = df.copy()
    
    # 1. 변환 대상 접두사 (절대값으로 존재하는 모든 데이터)
    prefixes = [
        'eth_', 'btc_', 'bnb_', 'xrp_', 'sol_', 'ada_', 'doge_', 'avax_', 'dot_', # 업비트 코인 가격
        'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_', 'l2_', # DeFi TVL
        'sp500_', 'gold_', 'dxy_', 'vix_', 'usdt_', 'fred_' # 매크로
    ]
    
    # 2. 보존해야 할 키워드 (이미 비율/변화율인 것들)
    stationary_keywords = [
        '_return', '_pct', '_ratio', '_change', '_spread', '_corr', '_beta', '_zscore', 
        '_trend', '_volatility', '_premium', '_implied', # [NEW] 김프 관련 파생 변수 보호
        'rsi_', 'cci_', 'adx_', 'atr_', 'obv', 'mfi_', 'willr_', 
        'target_', 'next_'
    ]
    
    # 3. 기술적 지표 중 절대값(가격)인 것들 (이동평균선, 볼린저밴드 등)
    ta_absolute_keywords = ['sma_', 'ema_', 'wma_', 'tema_', 'hma_', 'vwma_', 'bbu_', 'bbl_', 'bbm_', 'kc_']
    
    cols_to_drop = []
    
    print("   -> Transforming Non-Stationary Features...")
    
    for col in df_proc.columns:
        col_lower = col.lower()
        is_stationary = any(k in col_lower for k in stationary_keywords)
        
        # A. 접두사(eth_, btc_ 등)를 가진 원본 데이터
        if any(col_lower.startswith(p) for p in prefixes):
            if not is_stationary:
                # 변화율 & 이격도 생성 (중복 방지)
                if f'{col}_pct_change' not in df_proc.columns:
                    df_proc[f'{col}_pct_change'] = df_proc[col].pct_change().fillna(0)
                if f'{col}_trend_ratio' not in df_proc.columns:
                    ma_30 = df_proc[col].rolling(30).mean()
                    df_proc[f'{col}_trend_ratio'] = (df_proc[col] / (ma_30 + 1e-9) - 1).fillna(0)
                cols_to_drop.append(col)

        # B. 기술적 지표 중 절대값(가격 단위)
        elif any(ta_kw in col_lower for ta_kw in ta_absolute_keywords):
            if 'ETH_Close' in df_proc.columns:
                df_proc[f'{col}_ratio'] = (df_proc[col] / (df_proc['ETH_Close'] + 1e-9) - 1).fillna(0)
            cols_to_drop.append(col)
        
        # [NEW] C. 바이낸스 가격 데이터 명시적 삭제
        # 컬럼명에 '_binance_'가 포함되어 있고, Open/High/Low/Close/Volume이면 삭제
        elif '_binance_' in col_lower:
            if any(price_type in col_lower for price_type in ['open', 'high', 'low', 'close', 'volume']):
                # 이미 김프 계산에 사용되었으므로, 원본 USDT 가격은 필요 없음
                cols_to_drop.append(col)
    
    # [핵심] D. 업비트 원본 OHLCV도 삭제
    ohlcv_patterns = ['_Open', '_High', '_Low', '_Close', '_Volume']
    for col in df_proc.columns:
        if any(pattern in col for pattern in ohlcv_patterns):
            # 단, 이미 처리된 컬럼(_ratio, _change 등)은 제외
            if not any(k in col.lower() for k in stationary_keywords):
                cols_to_drop.append(col)
    
    # 중복 제거 및 date 보호
    cols_to_drop = list(set(cols_to_drop))
    if 'date' in cols_to_drop: 
        cols_to_drop.remove('date')
        
    print(f"      Dropping {len(cols_to_drop)} non-stationary columns")
    if len(cols_to_drop) > 0:
        print(f"      Examples: {cols_to_drop[:5]}")
    
    # 과감하게 삭제
    df_proc.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    
    # 무한대/NaN 처리
    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df_proc

def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """
    암호화폐 시계열 결측치 처리 (논문 기반: Forward Fill -> 0)
    
    Args:
        df_clean (pd.DataFrame): 처리할 데이터프레임
        train_start_date (str/datetime): 학습 시작 날짜 (Lookback 제외용)
        is_train (bool): 학습 데이터인지 여부 (True면 통계치 계산, False면 적용만)
        train_stats (dict): 학습 데이터에서 계산된 통계치 (테스트셋 적용용)
    
    Returns:
        pd.DataFrame: 결측치가 처리된 데이터프레임
        dict: 학습 데이터의 통계치 (is_train=True일 때만)
    """
    
    # 1. 날짜 필터링 (학습 시작일 이전 데이터 제거)
    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    
    # 원본 보존을 위해 복사
    df_proc = df_clean.copy()
    
    # 날짜 컬럼이 있으면 필터링 수행
    if 'date' in df_proc.columns:
        df_proc = df_proc[df_proc['date'] >= train_start_date].reset_index(drop=True)
    
    # 2. Feature 컬럼 선택 (타겟과 날짜 제외)
    target_cols = ['next_log_return', 'next_direction', 'next_close', 'next_open', 
                   'target_triple_barrier', 'target_next_return']
    exclude_cols = target_cols + ['date']
    feature_cols = [col for col in df_proc.columns if col not in exclude_cols]
    
    # 3. 결측치 처리 전략: Forward Fill -> 0
    # 시계열 데이터이므로 직전 값으로 채우는 것이 가장 합리적
    df_proc[feature_cols] = df_proc[feature_cols].fillna(method='ffill')
    df_proc[feature_cols] = df_proc[feature_cols].fillna(0)
    
    # 4. 무한대(inf) 처리
    # 0으로 나누기 등으로 생긴 무한대를 NaN으로 바꾸고 다시 처리
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan)
    df_proc[feature_cols] = df_proc[feature_cols].fillna(0)
    
    # 5. 최종 확인
    missing_count = df_proc[feature_cols].isnull().sum().sum()
    if missing_count > 0:
        # 그래도 남으면 0으로 강제 할당
        df_proc[feature_cols] = df_proc[feature_cols].fillna(0)
    
    if is_train:

        return df_proc, {}
    else:
        return df_proc

def run_feature_pipeline(df_raw):
    print("🚀 Starting FINAL Feature Engineering Pipeline...")
    
    # 1. 기술적 지표 & 파생 변수 생성 (여기선 원본 가격 필요)
    df = calculate_technical_indicators(df_raw)
    df = process_macro_features(df)
    df = process_onchain_features(df)
    df = add_enhanced_cross_crypto_features(df)
    df = process_kimp_features(df) 
    df = add_lag_and_volatility_features(df)
    
    print(f"✅ Feature Generation Complete. Shape: {df.shape}")
    

    df = create_targets(df)
    print(f"✅ Targets Created. Shape: {df.shape}")
    

    df = preprocess_non_stationary_features(df)
    print(f"✅ Non-Stationary Features Removed. Shape: {df.shape}")

    df = handle_missing_values_final(df)
    print(f"✅ Missing Values Handled. Final Shape: {df.shape}")
    
    return df


In [4]:
def select_features_multi_target(X_train, y_train, target_type='direction', top_n=30):
    
    atr_col_name = 'ATR_14'

    if target_type == 'direction':
        selected, stats = select_features_verified(
            X_train, 
            y_train['next_direction'], 
            task='class', 
            top_n=top_n
        )
        
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            selected.pop()
            selected.insert(0, atr_col_name)
    
    
    print(", ".join(selected))
    return selected, stats


def select_features_verified(X_train, y_train, task='class', top_n=30, verbose=True):
    
    if task == 'class':
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
    
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    if task == 'class':
        estimator = LGBMClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    else:
        estimator = LGBMRegressor(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42,
            verbose=-1
        )
    
    rfe = RFE(
        estimator=estimator,
        n_features_to_select=top_n,
        step=0.1,
        verbose=0
    )
    
    rfe.fit(X_train, y_train)
    rfe_features = X_train.columns[rfe.support_].tolist()

    if task == 'class':
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    else:
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
    
    rf_model.fit(X_train, y_train)
    rf_importances = rf_model.feature_importances_
    rf_idx = np.argsort(rf_importances)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]

    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0:
                    break
    
    return selected_features, {
        'mi_features': mi_features,
        'rfe_features': rfe_features,
        'rf_features': rf_features,
        'feature_votes': feature_votes,
        'mi_scores': dict(zip(X_train.columns, mi_scores)),
        'rf_importances': dict(zip(X_train.columns, rf_importances))
    }



def split_walk_forward_method(df, train_start_date, 
                              final_test_start='2025-01-01',
                              initial_train_size=800,    
                              val_size=150,             
                              test_size=150,           
                              step=150,                  
                              gap_size=7):
    """
    Reverse Rolling Walk-Forward Validation
    - 마지막 날짜부터 시작해서 과거로 rolling
    - Train 크기는 고정 (initial_train_size)
    - Final holdout은 2025-01-01부터 고정
    """
    
    df_period = df[df['date'] >= train_start_date].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    
    total_days = len(df_period)
    min_required_days = initial_train_size + val_size + (gap_size * 2) + test_size
    n_splits = (total_days - min_required_days) // step + 1
    
    print(f"\n{'='*80}")
    print(f"Reverse Rolling Walk-Forward Configuration ")
    print(f"{'='*80}")
    print(f"Total: {len(df_period)} days")
    print(f"Rolling train size: {initial_train_size} days (FIXED)")
    print(f"Val: {val_size} days | Test: {test_size} days")
    print(f"Gap: {gap_size} days | Step: {step} days (BACKWARD)")
    print(f"Target: {n_splits} walk-forward + 1 final holdout")
    print(f"{'='*80}\n")
    
    folds = []
    
    # 역방향 rolling
    for fold_idx in range(n_splits):
        test_end_idx = total_days - (fold_idx * step)
        test_start_idx = test_end_idx - test_size
        
        if test_start_idx < 0:
            break
        
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0:
            break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_idx': fold_idx + 1,
            'fold_type': 'walk_forward_rolling_reverse'
        })
    
    # 시간순으로 정렬
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
        print(f"Fold {fold['fold_idx']} (walk_forward_rolling)")
        print(f"  Train: {len(fold['train']):4d}d  {fold['train']['date'].min().date()} ~ {fold['train']['date'].max().date()}")
        print(f"  Val:   {len(fold['val']):4d}d  {fold['val']['date'].min().date()} ~ {fold['val']['date'].max().date()}")
        print(f"  Test:  {len(fold['test']):4d}d  {fold['test']['date'].min().date()} ~ {fold['test']['date'].max().date()}\n")
    
    # Final holdout
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        
        final_val_end_idx = len(pre_final_df)
        final_val_start_idx = final_val_end_idx - val_size
        final_train_end_idx = final_val_start_idx - gap_size
        final_train_start_idx = final_train_end_idx - initial_train_size
        
        if final_train_start_idx < 0:
            final_train_start_idx = 0
        
        final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
        final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
        
        print(f"Fold {len(folds) + 1} (final_holdout)")
        print(f"  Train: {len(final_train_data):4d}d  {final_train_data['date'].min().date()} ~ {final_train_data['date'].max().date()}")
        print(f"  Val:   {len(final_val_data):4d}d  {final_val_data['date'].min().date()} ~ {final_val_data['date'].max().date()}")
        print(f"  Test:  {len(final_test_df):4d}d  {final_test_df['date'].min().date()} ~ {final_test_df['date'].max().date()}\n")
        
        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })
    
    print(f"{'='*80}")
    print(f"Created {len(folds)} folds total")
    print(f"{'='*80}\n")
    
    return folds


def process_single_split(split_data, target_type='direction', top_n=40, fold_idx=None):
    """
    각 fold를 독립적으로 처리 (feature selection 포함)
    """
    
    train_df = split_data['train']
    val_df = split_data['val']
    test_df = split_data['test']
    fold_type = split_data.get('fold_type', 'unknown')
    
    if fold_idx is not None:
        print(f"\n{'='*60}")
        print(f"Processing Fold {fold_idx} ({fold_type})")
        print(f"{'='*60}")
    

    train_processed, missing_stats = handle_missing_values_paper_based(
        train_df.copy(),
        train_start_date=train_df['date'].min(),
        is_train=True
    )
    
    val_processed = handle_missing_values_paper_based(
        val_df.copy(),
        train_start_date=val_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    test_processed = handle_missing_values_paper_based(
        test_df.copy(),
        train_start_date=test_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    target_cols = ['next_direction','next_log_return', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price']
    
    train_processed = train_processed.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_processed.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_processed.dropna(subset=target_cols).reset_index(drop=True)

    feature_cols = [col for col in train_processed.columns 
                   if col not in target_cols + ['date']]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    print(f"\n[Feature Selection for Fold {fold_idx}]")
    print(f"Training data shape: {X_train.shape}")
    
    selected_features, selection_stats = select_features_multi_target(
        X_train, 
        y_train, 
        target_type=target_type, 
        top_n=top_n
    )
    
    print(f"Selected {len(selected_features)} features for this fold")
    
    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]
    
    robust_scaler = RobustScaler()
    standard_scaler = StandardScaler()
    
    X_train_robust = robust_scaler.fit_transform(X_train_sel)
    X_val_robust = robust_scaler.transform(X_val_sel)
    X_test_robust = robust_scaler.transform(X_test_sel)
    
    X_train_standard = standard_scaler.fit_transform(X_train_sel)
    X_val_standard = standard_scaler.transform(X_val_sel)
    X_test_standard = standard_scaler.transform(X_test_sel)
    
    print(f"Scaling completed for Fold {fold_idx}")
    print(f"{'='*60}\n")
    print(f"\n--- [Verification] Actual Date Ranges for Fold {fold_idx} (Post-dropna) ---")
    if not train_processed.empty:
        print(f"  Train (Actual): {len(train_processed):4d} ({train_processed['date'].min().date()} ~ {train_processed['date'].max().date()})")
    if not val_processed.empty:
        print(f"  Val   (Actual): {len(val_processed):4d} ({val_processed['date'].min().date()} ~ {val_processed['date'].max().date()})")
    if not test_processed.empty:
        print(f"  Test  (Actual): {len(test_processed):4d} ({test_processed['date'].min().date()} ~ {test_processed['date'].max().date()})")
    print(f"---------------------------------------------------------------------\n")
    
    result = {
        'train': {
            'X_robust': X_train_robust,
            'X_standard': X_train_standard,
            'X_raw': X_train_sel,
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_robust,
            'X_standard': X_val_standard,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X_robust': X_test_robust,
            'X_standard': X_test_standard,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'scaler': robust_scaler, 
        'stats': {
            'robust_scaler': robust_scaler,
            'standard_scaler': standard_scaler,
            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'missing_stats': missing_stats,  
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx
        }
    }
    
    return result



def build_complete_pipeline_corrected(df_raw, train_start_date, 
                                     final_test_start='2025-01-01',
                                     method='tvt', target_type='direction',lookahead_candles=8,atr_multiplier_profit=1.5, 
                                     atr_multiplier_stop=1.0, **kwargs):
    """
    전체 파이프라인 실행 함수
    
    Parameters:
    -----------
    df_raw : DataFrame
        원본 데이터
    train_start_date : str
        학습 데이터 시작 날짜
    final_test_start : str, default='2025-01-01'
        최종 고정 테스트 시작 날짜
        - TVT: 이 날짜부터 마지막까지 테스트
        - Walk-forward: 이 날짜 이전은 walk-forward folds, 이후는 final holdout
    method : str, default='tvt'
        'tvt' 또는 'walk_forward'
    target_type : str, default='direction'
        'direction', 'return', 'price', 'direction_return', 'direction_price'
    **kwargs : dict
        각 method에 필요한 추가 파라미터
    """
    
    df = df_raw.copy()

    df= run_feature_pipeline(df_raw)

    pd.set_option('display.max_columns', None)
    df = df.iloc[:-lookahead_candles]  
    
    
    
    split_kwargs = {}
    

    split_kwargs['final_test_start'] = final_test_start
    if 'n_splits' in kwargs:
        split_kwargs['n_splits'] = kwargs['n_splits']
    if 'initial_train_size' in kwargs:
        split_kwargs['initial_train_size'] = kwargs['initial_train_size']
    if 'test_size' in kwargs:
        split_kwargs['test_size'] = kwargs['test_size']
    if 'val_size' in kwargs:
        split_kwargs['val_size'] = kwargs['val_size']
    if 'step' in kwargs:
        split_kwargs['step'] = kwargs['step']
    splits = split_walk_forward_method(df, train_start_date, **split_kwargs)

    result = [
        process_single_split(
            fold, 
            target_type=target_type,  
            top_n=30,
            fold_idx=fold['fold_idx']
        ) 
        for fold in splits
    ]
    
    return result


In [5]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

class DirectionModels:
    
    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 150, 400),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 15, 50),
                'subsample': trial.suggest_float('subsample', 0.5, 0.8),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 20.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 20.0, log=True),
                'min_child_samples': trial.suggest_int('min_child_samples', 50, 100),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0, log=True),
                'min_split_gain': trial.suggest_float('min_split_gain', 0.01, 1.0, log=True),
                'path_smooth': trial.suggest_float('path_smooth', 0.0, 1.0),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.8),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.8),
                'bagging_freq': 1,
                'class_weight': 'balanced',
                'random_state': 42,
                'verbose': -1,
                'force_col_wise': True
            }

            model = LGBMClassifier(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='binary_logloss',
                callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
            )

            train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
            train_acc = accuracy_score(y_train, train_pred)
            val_acc = accuracy_score(y_val, y_val_pred)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty

        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=8),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=10)
        )

        study.optimize(objective, n_trials=30, show_progress_bar=False)

        best_params = study.best_params
        best_params.update({
            'class_weight': 'balanced',
            'random_state': 42,
            'verbose': -1,
            'force_col_wise': True,
            'bagging_freq': 1
        })

        final_model = LGBMClassifier(**best_params)
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
        )

        train_pred = final_model.predict(X_train)
        val_pred = final_model.predict(X_val)
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"[LightGBM] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")

        return final_model
    
    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        neg_count = (y_train == 0).sum()
        pos_count = (y_train == 1).sum()
        base_scale_pos_weight = neg_count / max(pos_count, 1)

        def objective(trial):
            scale_multiplier = trial.suggest_float('scale_multiplier', 0.8, 1.2)
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 150, 400),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 0.8),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 0.8),
                'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 0.8),
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 20.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 2.0, 20.0, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 10, 30),
                'gamma': trial.suggest_float('gamma', 0.1, 2.0, log=True),
                'max_delta_step': trial.suggest_float('max_delta_step', 0, 3),
                'scale_pos_weight': base_scale_pos_weight * scale_multiplier,
                'random_state': 42,
                'n_jobs': -1,
                'tree_method': 'hist',
                'eval_metric': 'logloss'
            }

            model = XGBClassifier(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                verbose=False
            )

            train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
            train_acc = accuracy_score(y_train, train_pred)
            val_acc = accuracy_score(y_val, y_val_pred)
            
            gap_penalty = max(0, (train_acc - val_acc) - 0.03)
            return val_acc - 1.0 * gap_penalty

        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=8),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=10)
        )

        study.optimize(objective, n_trials=30, show_progress_bar=False)

        best_params = study.best_params
        best_params.update({
            'scale_pos_weight': base_scale_pos_weight * best_params['scale_multiplier'],
            'random_state': 42,
            'n_jobs': -1,
            'tree_method': 'hist',
            'eval_metric': 'logloss'
        })
        del best_params['scale_multiplier']

        final_model = XGBClassifier(**best_params)
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        train_pred = final_model.predict(X_train)
        val_pred = final_model.predict(X_val)
        train_acc = accuracy_score(y_train, train_pred)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"[XGBoost] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f} | ScalePosWeight: {best_params['scale_pos_weight']:.2f}")

        return final_model

    
    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            param = {
                'iterations': trial.suggest_int('iterations', 100, 300),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'depth': trial.suggest_int('depth', 2, 4),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 8.0, 20.0),
                'subsample': trial.suggest_float('subsample', 0.4, 0.7),
                'rsm': trial.suggest_float('rsm', 0.4, 0.7),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 40, 80),
                'auto_class_weights': 'Balanced',
                'random_seed': 42,
                'verbose': False,
                'early_stopping_rounds': 20
            }
            
            model = CatBoostClassifier(**param)
            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                verbose=False
            )
            
            val_acc = model.score(X_val, y_val)
            return val_acc
        
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=10),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=15)
        )
        
        study.optimize(objective, n_trials=25, show_progress_bar=False)
        
        model = CatBoostClassifier(**study.best_params, auto_class_weights='Balanced', random_seed=42, verbose=False)
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            verbose=False
        )
        
        train_acc = model.score(X_train, y_train)
        val_acc = model.score(X_val, y_val)
        print(f"[CatBoost] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Gap: {train_acc - val_acc:.4f}")
        
        return model


  


In [6]:
# ML Models (19개)
ML_MODELS_CLASSIFICATION = [
    #{'index': 1, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 2, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 3, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True},
   # {'index': 5, 'name': 'LogisticRegression', 'func': DirectionModels.logistic_regression, 'needs_val': True},
    #{'index': 8, 'name': 'AdaBoost', 'func': DirectionModels.adaboost, 'needs_val': True},
    {'index': 9, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
   # {'index': 13, 'name': 'GradientBoosting', 'func': DirectionModels.gradient_boosting, 'needs_val': True},
    #{'index': 14, 'name': 'HistGradientBoosting', 'func': DirectionModels.histgradient_boosting, 'needs_val': True},
    #{'index': 15, 'name': 'StackingEnsemble', 'func': DirectionModels.stacking_ensemble, 'needs_val': True},
]

# DL Models (11개)
DL_MODELS_CLASSIFICATION = [
    #{'index': 19, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
    #{'index': 20, 'name': 'BiLSTM', 'func': DirectionModels.bilstm, 'needs_val': True},
    #{'index': 21, 'name': 'GRU', 'func': DirectionModels.gru, 'needs_val': True}
]


In [7]:
import os
import gc
import time
import joblib
import traceback
import shap
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

# ============================================================================
# 1. 평가 및 저장 클래스 (핵심: 모든 Artifacts 저장)
# ============================================================================
class ModelEvaluator:
    def __init__(self, experiment_name, fold_idx, base_dir='model_results'):
        self.results = []
        # 저장 경로: model_results/20251203/experiment_name/fold_1/
        timestamp = datetime.now().strftime("%Y%m%d")
        self.save_dir = os.path.join(base_dir, timestamp, experiment_name, f"fold_{fold_idx}")
        os.makedirs(self.save_dir, exist_ok=True)
        print(f"📁 Artifacts will be saved to: {self.save_dir}")

    def save_scaler_and_features(self, scaler, feature_names):
        """[필수] 실전 매매를 위한 Scaler와 Feature Names 저장"""
        joblib.dump(scaler, os.path.join(self.save_dir, 'scaler.pkl'))
        joblib.dump(feature_names, os.path.join(self.save_dir, 'feature_names.pkl'))

    def evaluate_and_save(self, model, X_test, y_test_df, test_dates, model_name, feature_names):
        print(f"\n🔎 Evaluating {model_name}...")
        
        # 1. 예측 (확률 추출)
        try:
            pred_proba = model.predict_proba(X_test)[:, 1]
        except AttributeError:
            print(f"⚠️ {model_name} has no predict_proba. Skipping.")
            return None

        y_true = y_test_df['next_direction'].values
        
        # 2. 트레이딩 관점 평가 (Threshold별 승률)
        trading_metrics = {}
        thresholds = [0.5, 0.55, 0.6, 0.65, 0.7]
        
        print(f"   -> [Trading Metrics]")
        for th in thresholds:
            mask = pred_proba >= th
            count = mask.sum()
            if count > 0:
                win_rate = np.mean(y_true[mask] == 1)
                print(f"      TH {th:.2f}: Trades={count}, WinRate={win_rate:.2%}")
            else:
                win_rate = 0.0
            
            trading_metrics[f'WinRate_{th}'] = win_rate
            trading_metrics[f'Trades_{th}'] = count

        # 3. 기본 메트릭
        acc = accuracy_score(y_true, (pred_proba >= 0.5).astype(int))
        auc = roc_auc_score(y_true, pred_proba)
        
        result = {
            'Model': model_name,
            'Accuracy': acc,
            'AUC': auc,
            **trading_metrics
        }
        self.results.append(result)

        # 4. [저장 1] 백테스팅용 Prediction CSV
        self._save_prediction_csv(model_name, pred_proba, y_test_df, test_dates)
        
        # 5. [저장 2] 모델 객체 (PKL)
        joblib.dump(model, os.path.join(self.save_dir, f'model_{model_name}.pkl'))
        
        # 6. [저장 3] SHAP 데이터 (분석용)
        self._save_shap(model, X_test, model_name, feature_names)
        
        return result

    def _save_prediction_csv(self, model_name, pred_proba, y_test_df, dates):
        df_pred = pd.DataFrame({
            'date': dates,
            'actual_direction': y_test_df['next_direction'].values,
            'actual_return': y_test_df['next_log_return'].values,
            'close_price': y_test_df['ETH_Close'].values if 'ETH_Close' in y_test_df.columns else 0,
            'pred_proba': pred_proba,
            'confidence': np.abs(pred_proba - 0.5) * 2
        })
        df_pred.to_csv(os.path.join(self.save_dir, f'pred_{model_name}.csv'), index=False)

    def _save_shap(self, model, X_test, model_name, feature_names):
        try:
            # 속도를 위해 최근 300개만 샘플링
            X_sample = X_test[-300:] if len(X_test) > 300 else X_test
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_sample)
            
            if isinstance(shap_values, list):
                shap_values = shap_values[1]
                
            df_shap = pd.DataFrame(shap_values, columns=feature_names)
            df_shap.to_csv(os.path.join(self.save_dir, f'shap_{model_name}.csv'), index=False)
            
            # Feature Importance
            imp = np.abs(df_shap).mean().sort_values(ascending=False)
            imp.to_csv(os.path.join(self.save_dir, f'importance_{model_name}.csv'))
        except Exception as e:
            print(f"⚠️ SHAP failed: {e}")

    def save_summary(self):
        pd.DataFrame(self.results).to_csv(os.path.join(self.save_dir, 'summary_metrics.csv'), index=False)



class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    @staticmethod
    def create_sequences(X, y, lookback):
        Xs, ys = [], []
        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y.iloc[i] if hasattr(y, 'iloc') else y[i])
        X_arr = np.array(Xs)
        y_arr = np.array(ys)
        del Xs, ys
        gc.collect()
        return X_arr, y_arr
    
    @staticmethod
    def clear_memory():
        keras.backend.clear_session()
        try:
            tf.compat.v1.reset_default_graph()
        except:
            pass
        gc.collect()
        time.sleep(0.5)
    
    def train_ml_model(self, model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates, task='classification'):
        model = None
        try:
            if model_config.get('needs_val', False):
                model = model_config['func'](X_train, y_train, X_val, y_val)
            else:
                model = model_config['func'](X_train, y_train)
            
            is_mlp = (model_config['name'] == 'MLP')
            
            self.evaluator.evaluate_classification_model(
                model, X_train, y_train, X_val, y_val, X_test, y_test_df,
                test_dates, model_config['name'], is_deep_learning=is_mlp
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                if is_mlp:
                    self.clear_memory()
                else:
                    gc.collect()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            if model_config.get('name') == 'MLP':
                self.clear_memory()
            else:
                gc.collect()
    
    def train_dl_model(self, model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_df_seq, test_dates_seq, input_shape, task='classification'):
        model = None
        try:
            self.clear_memory()
            
            model = model_config['func'](X_train_seq, y_train_seq, X_val_seq, y_val_seq, input_shape)
            
            self.evaluator.evaluate_classification_model(
                model, X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                X_test_seq, y_test_df_seq, test_dates_seq,
                model_config['name'], is_deep_learning=True
            )
            
            if not self.evaluator.save_models:
                del model
                model = None
                self.clear_memory()
            
            return True
        except Exception as e:
            print(f"    {model_config['name']} failed: {type(e).__name__}")
            return False
        finally:
            if model is not None and not self.evaluator.save_models:
                try:
                    del model
                except:
                    pass
            self.clear_memory()


def train_all_models(X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates, evaluator, lookback=30, ml_models=None, dl_models=None):
    trainer = ModelTrainer(evaluator, lookback)

    ml_success = 0
    for model_config in ml_models:
        if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates):
            ml_success += 1
        gc.collect()
    
    trainer.clear_memory()
    
    y_test_direction = y_test_df['next_direction'].values

    X_train_seq, y_train_seq = trainer.create_sequences(X_train, y_train, lookback)
    X_val_seq, y_val_seq = trainer.create_sequences(X_val, y_val, lookback)
    X_test_seq, y_test_seq = trainer.create_sequences(X_test, y_test_direction, lookback)
    
    test_dates_seq = test_dates[lookback:]
    y_test_df_seq = y_test_df.iloc[lookback:].reset_index(drop=True)
    
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    
    dl_success = 0
    for model_config in dl_models:
        trainer.clear_memory()
        
        if model_config['name'] in ['TabNet', 'StackingEnsemble', 'VotingHard', 'VotingSoft']:
            if trainer.train_ml_model(model_config, X_train, y_train, X_val, y_val, X_test, y_test_df, test_dates):
                dl_success += 1
        else:
            if trainer.train_dl_model(model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_df_seq, test_dates_seq, input_shape):
                dl_success += 1
        
        gc.collect()
    
    del X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, y_test_df_seq, test_dates_seq
    trainer.clear_memory()
    
    return ml_success + dl_success

In [8]:
lookahead_options = [5] # 예: 5일 뒤 예측
atr_profit_multipliers = [2.0] # 예: 2.0 ATR 익절
atr_stop_multipliers = [1.0]   # 예: 1.0 ATR 손절

print("🚀 Starting Experiments...")

for lookahead in lookahead_options:
    for p_mult in atr_profit_multipliers:
        for s_mult in atr_stop_multipliers:
            
            # 실험 이름 생성
            exp_name = f"L{lookahead}_TP{p_mult}_SL{s_mult}"
            print(f"\n{'='*60}\n[Experiment] {exp_name}\n{'='*60}")

            try:

                pipeline_result = build_complete_pipeline_corrected(
                    df_final, 
                    Config.TRAIN_START_DATE,
                    method='walk_forward',
                    target_type='direction',
                    final_test_start='2025-01-01',
                    lookahead_candles=lookahead,
                    atr_multiplier_profit=p_mult,
                    atr_multiplier_stop=s_mult
                )

                # 2. Fold별 학습 및 평가
                for fold_idx, fold_data in enumerate(pipeline_result, start=1):
                    print(f"\n>>> Processing Fold {fold_idx}/{len(pipeline_result)}")
                    
                    # 데이터 추출
                    X_train = fold_data['train']['X_robust']
                    y_train = fold_data['train']['y']['next_direction'] # 타겟 컬럼명 확인
                    X_val   = fold_data['val']['X_robust']
                    y_val   = fold_data['val']['y']['next_direction']
                    X_test  = fold_data['test']['X_robust']
                    y_test_df = fold_data['test']['y'] # 전체 데이터프레임 (가격 등 포함)
                    test_dates = fold_data['test']['dates']
                    
                    # Feature Names 복구 (SHAP용)
                    # build_pipeline에서 feature_names를 저장해두었다면 사용, 아니면 임시 생성
                    feature_names = fold_data.get('feature_names', [f'feat_{i}' for i in range(X_train.shape[1])])

                    # 3. 평가 객체 생성 (폴더 자동 생성)
                    evaluator = ModelEvaluator(exp_name, fold_idx)
                    
                    if 'scaler' in fold_data:
                        evaluator.save_scaler_and_features(fold_data['scaler'], feature_names)
                    
                    # 4. 모델 학습 및 저장
                    for model_config in ML_MODELS_CLASSIFICATION:
                        name = model_config['name']
                        func = model_config['func']
                        
                        print(f"   Training {name}...")
                        # 학습
                        model = func(X_train, y_train, X_val, y_val)
                        
                        # 평가 및 저장 (Evaluation + Saving)
                        evaluator.evaluate_and_save(
                            model, X_test, y_test_df, test_dates, name, feature_names
                        )
                        
                        del model
                        gc.collect()
                    
                    # Fold 요약 저장
                    evaluator.save_summary()
                    
            except Exception as e:
                print(f"❌ Experiment {exp_name} Failed: {e}")
                traceback.print_exc()
                
print("\n✅ All Experiments Completed Successfully.")

🚀 Starting Experiments...

[Experiment] L5_TP2.0_SL1.0
🚀 Starting FINAL Feature Engineering Pipeline...
✅ Feature Generation Complete. Shape: (2362, 198)
✅ Targets Created. Shape: (2362, 201)
   -> Transforming Non-Stationary Features...
      Dropping 117 non-stationary columns
      Examples: ['SMA_50', 'ETH_Open', 'fred_CPI', 'BBU_20', 'ADA_binance_High']
✅ Non-Stationary Features Removed. Shape: (2362, 308)
✅ Missing Values Handled. Final Shape: (2362, 308)

Reverse Rolling Walk-Forward Configuration 
Total: 2157 days
Rolling train size: 800 days (FIXED)
Val: 150 days | Test: 150 days
Gap: 7 days | Step: 150 days (BACKWARD)
Target: 7 walk-forward + 1 final holdout

Fold 1 (walk_forward_rolling)
  Train:  800d  2020-05-23 ~ 2022-07-31
  Val:    150d  2022-08-08 ~ 2023-01-04
  Test:   150d  2023-01-12 ~ 2023-06-10

Fold 2 (walk_forward_rolling)
  Train:  800d  2020-10-20 ~ 2022-12-28
  Val:    150d  2023-01-05 ~ 2023-06-03
  Test:   150d  2023-06-11 ~ 2023-11-07

Fold 3 (walk_forward

Traceback (most recent call last):
  File "/tmp/ipykernel_31522/1803291370.py", line 17, in <module>
    pipeline_result = build_complete_pipeline_corrected(
  File "/tmp/ipykernel_31522/4144229175.py", line 398, in build_complete_pipeline_corrected
    result = [
  File "/tmp/ipykernel_31522/4144229175.py", line 399, in <listcomp>
    process_single_split(
  File "/tmp/ipykernel_31522/4144229175.py", line 254, in process_single_split
    train_processed = train_processed.dropna(subset=target_cols).reset_index(drop=True)
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/pandas/core/frame.py", line 6561, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['next_direction', 'next_log_return', 'next_close', 'next_open', 'take_profit_price', 'stop_loss_price']


In [9]:
try:
    run_and_save_master_summary(RESULT_DIR)
except Exception as e:
    print(f"Master summary failed: {e}")

print(f"\nSuccessful: {len(successful_experiments)}, Failed: {len(failed_experiments)}")
print(f"Results: {RESULT_DIR}")

if failed_experiments:
    print("\nFailed experiments:")
    for exp_name, error in failed_experiments:
        print(f"  {exp_name}: {error}")

Master summary failed: name 'run_and_save_master_summary' is not defined


NameError: name 'successful_experiments' is not defined